In [ ]:
!pip install tensorboard
!pip install gensim
!pip install pyLDAvis
!pip install wordcloud matplotlib
!pip install emoji
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gzip -d cc.en.300.bin.gz
!pip install fasttext

In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import re
import nltk
from nltk.corpus import stopwords
from collections import defaultdict, Counter
import pandas as pd
import emoji
from gensim.models import Word2Vec
import time
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, precision_recall_curve, auc
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

In [ ]:
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv

--2025-03-19 13:01:12--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.170.207, 142.251.175.207, 74.125.24.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.170.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘data/full_dataset/goemotions_1.csv.1’

goemotions_1.csv.1  100%[===================>]  13.52M  7.63MB/s    in 1.8s    

2025-03-19 13:01:14 (7.63 MB/s) - ‘data/full_dataset/goemotions_1.csv.1’ saved [14174600/14174600]

--2025-03-19 13:01:14--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.170.207, 142.251.175.207, 74.125.24.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.170.207|:443... connected.
HTTP request sent, awaiting r

In [ ]:
goemotions_1 = pd.read_csv('data/full_dataset/goemotions_1.csv')
goemotions_2 = pd.read_csv('data/full_dataset/goemotions_2.csv')
goemotions_3 = pd.read_csv('data/full_dataset/goemotions_3.csv')

combined_df = pd.concat([goemotions_1, goemotions_2, goemotions_3], ignore_index=True)
emotion_columns = combined_df.columns[9:]




combined_df = pd.read_csv('/content/emotional_data.csv')

combined_df['created_utc'] = pd.to_datetime(combined_df['created_utc'], unit='s', errors='coerce')
combined_df.to_csv('data/full_dataset/goemotions_combined.csv', index=False)
combined_df = combined_df.query("example_very_unclear != True")

#work white emoji...

In [ ]:

# Dictionary to store emojis with their positions and texts
emoji_dict = {}

# Function to extract emojis and build the dictionary
def build_emoji_dict(df):
    for idx, row in df.iterrows():
        text = row['text']
        for char in text:
            if emoji.is_emoji(char):
                if char not in emoji_dict:
                    emoji_dict[char] = []
                emoji_dict[char].append((idx, text))
build_emoji_dict(combined_df)


#Process

In [ ]:
def preprocess_text(text):
    text = re.sub(r"[<>]", "", text)
    text = f"[START] {text} [END]"

    text = re.sub(r'\[(\w+)\]', r'<\1>', text)
    text = emoji.demojize(text)
    text = re.sub(r"[^a-zA-Z0-9<> ]", "", text)

    text = text.lower()
    return text

combined_df['processed_text'] = combined_df['text'].apply(preprocess_text)

print(combined_df[['text', 'processed_text']].head())

                                                text  \
0     You do right, if you don't care then fuck 'em!   
1  Right? Considering it’s such an important docu...   
2  He isn't as big, but he's still quite popular....   
3                                that's adorable asf   
4  "Sponge Blurb Pubs Quaw Haha GURR ha AAa!" fin...   

                                      processed_text  
0  <start> you do right if you dont care then fuc...  
1  <start> right considering its such an importan...  
2  <start> he isnt as big but hes still quite pop...  
3                   <start> thats adorable asf <end>  
4  <start> sponge blurb pubs quaw haha gurr ha aa...  


In [ ]:
def create_mappings(df):
    word_to_index = {}
    index_to_word = {}
    index_counter = 1

    for text in df['processed_text']:
        words = text.split()

        for word in words:
            if word not in word_to_index:
                word_to_index[word] = index_counter
                index_to_word[index_counter] = word
                index_counter += 1

    return word_to_index, index_to_word


word_to_index, index_to_word = create_mappings(combined_df)

print("Word to Index Mapping:", dict(list(word_to_index.items())[:10]))
print("Index to Word Mapping:", dict(list(index_to_word.items())[:10]))

Word to Index Mapping: {'<start>': 1, 'you': 2, 'do': 3, 'right': 4, 'if': 5, 'dont': 6, 'care': 7, 'then': 8, 'fuck': 9, 'em': 10}
Index to Word Mapping: {1: '<start>', 2: 'you', 3: 'do', 4: 'right', 5: 'if', 6: 'dont', 7: 'care', 8: 'then', 9: 'fuck', 10: 'em'}


In [ ]:
len(word_to_index),len(index_to_word)

(24940, 24940)

In [ ]:
def text_to_index_sequence(text, word_to_index):
    words = text.split()
    index_sequence = [word_to_index[word] for word in words if word in word_to_index]
    return index_sequence
combined_df['text_as_indexes'] = combined_df['processed_text'].apply(lambda x: text_to_index_sequence(x, word_to_index))
print(combined_df[['text', 'processed_text', 'text_as_indexes']].head())

                                                text  \
0     You do right, if you don't care then fuck 'em!   
1  Right? Considering it’s such an important docu...   
2  He isn't as big, but he's still quite popular....   
3                                that's adorable asf   
4  "Sponge Blurb Pubs Quaw Haha GURR ha AAa!" fin...   

                                      processed_text  \
0  <start> you do right if you dont care then fuc...   
1  <start> right considering its such an importan...   
2  <start> he isnt as big but hes still quite pop...   
3                   <start> thats adorable asf <end>   
4  <start> sponge blurb pubs quaw haha gurr ha aa...   

                                     text_as_indexes  
0             [1, 2, 3, 4, 5, 2, 6, 7, 8, 9, 10, 11]  
1  [1, 4, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,...  
2  [1, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41...  
3                                [1, 50, 51, 52, 11]  
4  [1, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63..

#word2Vec

In [ ]:
def prepare_data_for_word2vec(df):
    processed_texts = df['processed_text'].apply(lambda x: x.split())
    return processed_texts

sentences = prepare_data_for_word2vec(combined_df)
model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)
word_vectors = {word: model.wv[word] for word in model.wv.index_to_key}
print("Word vector for 'start':", word_vectors['start'])

Word vector for 'start': [ 1.43997356e-01  9.80936646e-01 -3.22280437e-01  2.49828976e-02
 -8.49179849e-02 -2.26509571e-01  3.45500469e-01  8.84381831e-01
  1.90539196e-01 -1.47458106e-01  3.29569787e-01 -2.74357289e-01
  8.49218853e-03 -1.27615020e-01 -9.95273367e-02 -3.96747649e-01
  6.29019290e-02  1.94452032e-01  1.89422190e-01  1.04487449e-01
 -2.46406734e-01 -3.04845702e-02  1.73779115e-01  4.80746686e-01
 -1.16303109e-01  4.81422752e-01 -8.25350106e-01  3.59569751e-02
  1.84139535e-01 -3.84169847e-01 -1.22533785e-02 -6.15075648e-01
 -2.55221367e-01 -8.56387690e-02 -4.78570610e-02 -9.59383324e-02
  4.25008655e-01 -6.53701961e-01  2.58525908e-01 -9.08055827e-02
 -1.70327172e-01 -3.63490015e-01  4.46965426e-01 -4.13592875e-01
  6.71687782e-01  3.68146479e-01  2.58491069e-01 -1.95627138e-01
 -1.15605950e-01  3.02469552e-01  1.38134032e-01 -1.73135892e-01
 -2.81258196e-01  2.55588908e-02 -1.56331405e-01  8.43905434e-02
  9.14063752e-02  1.96273342e-01  1.28584847e-01  6.05602004e-02


#FestText

In [ ]:
import fasttext
fasttext_model = fasttext.load_model('/content/cc.en.300.bin')

def prepare_data_for_fasttext(df):
    processed_texts = df['processed_text'].apply(lambda x: x.split())
    return processed_texts

sentences = prepare_data_for_fasttext(combined_df)

word_vectors = {word: fasttext_model.get_word_vector(word) for sentence in sentences for word in sentence}

print("Word vector for 'start':", word_vectors.get('start', 'Word not found'))


Word vector for 'start': [ 7.91064836e-03 -9.43343155e-03  2.28888560e-02  1.15760624e-01
 -5.44784889e-02  3.52381244e-02  1.51470797e-02  9.68480669e-03
  3.72007266e-02 -8.26112553e-03 -3.63847502e-02  7.02371597e-02
 -1.53566375e-02  1.11012265e-01  1.33964136e-01  6.41972050e-02
  6.07942492e-02 -1.30296145e-02  1.28229149e-04  9.35186446e-02
 -6.10206239e-02 -2.02715285e-02 -1.50931003e-02 -9.10209268e-02
  1.57276485e-02 -1.85630284e-02 -6.97859824e-02  6.44239336e-02
 -4.34500724e-02  3.08100097e-02  2.29679141e-03 -5.58396354e-02
 -6.58368096e-02 -6.34431988e-02  3.13772000e-02  2.18865033e-02
  3.18698958e-02  1.05855837e-01 -2.08669528e-03 -9.21282396e-02
  3.49404141e-02 -9.22180153e-03  8.44760537e-02  6.59895763e-02
 -3.71814519e-02  7.32254684e-02  3.71225476e-02 -6.29411414e-02
 -3.49417143e-02 -3.72454687e-03  1.67517923e-02  1.74221359e-02
  8.67096111e-02 -5.21334670e-02 -9.95707512e-03 -1.62409935e-02
  3.27702239e-02  3.58919427e-03 -8.23421478e-02 -1.64883211e-03


#work white the labell

In [ ]:
from sklearn.model_selection import train_test_split

def split_data_for_emotion(combined_df, emotion_columns, emotion, test_size=0.2, random_state=42):
    # Separate the entries for the specific emotion
    emotion_entries = combined_df[combined_df[emotion] == 1].copy()

    # Set other emotion columns to 0
    for col in emotion_columns:
        if col != emotion:
            emotion_entries[col] = 0

    # Combine non-specific emotion entries
    all_other_emotions = pd.concat([combined_df[combined_df[e] == 1] for e in emotion_columns if e != emotion])

    # Handle potential list-type columns
    for col in all_other_emotions.columns:
        if all_other_emotions[col].apply(lambda x: isinstance(x, list)).any():
            all_other_emotions[col] = all_other_emotions[col].apply(lambda x: str(x) if isinstance(x, list) else x)

    # Remove duplicates
    all_other_emotions = all_other_emotions.drop_duplicates()

    # Split data into train and test
    train_emotion, test_emotion = train_test_split(emotion_entries, test_size=test_size, random_state=random_state)
    train_other, test_other = train_test_split(all_other_emotions, test_size=test_size, random_state=random_state)

    # Filter to keep only processed_text and current emotion
    train_emotion_filtered = train_emotion[['processed_text', emotion]]
    test_emotion_filtered = test_emotion[['processed_text', emotion]]
    train_other_filtered = train_other[['processed_text', emotion]]
    test_other_filtered = test_other[['processed_text', emotion]]

    # Combine emotion and non-emotion entries
    train_combined = pd.concat([train_emotion_filtered, train_other_filtered], ignore_index=True)
    test_combined = pd.concat([test_emotion_filtered, test_other_filtered], ignore_index=True)

    print(f"{emotion} - Train: {len(train_combined)}, Test: {len(test_combined)}")

    return train_combined, test_combined

# Apply for all emotions
emotion_data = {}
for emotion in emotion_columns:
    if emotion in combined_df.columns:
        train, test = split_data_for_emotion(combined_df, emotion_columns, emotion)
        emotion_data[emotion] = {'train': train, 'test': test}
    else:
        print(f"Emotion {emotion} not found in the dataset.")

print("Data splitting complete for all emotions!")


admiration - Train: 40942, Test: 10237
amusement - Train: 39793, Test: 9949
anger - Train: 39676, Test: 9919
annoyance - Train: 40856, Test: 10215
approval - Train: 41408, Test: 10353
caring - Train: 39932, Test: 9983
confusion - Train: 39764, Test: 9942
curiosity - Train: 40650, Test: 10164
desire - Train: 39538, Test: 9885
disappointment - Train: 40091, Test: 10024
disapproval - Train: 40248, Test: 10063
disgust - Train: 39722, Test: 9931
embarrassment - Train: 39174, Test: 9795
excitement - Train: 39564, Test: 9891
fear - Train: 39083, Test: 9772
gratitude - Train: 39913, Test: 9979
grief - Train: 38890, Test: 9724
joy - Train: 39741, Test: 9937
love - Train: 39618, Test: 9906
nervousness - Train: 39034, Test: 9760
optimism - Train: 40684, Test: 10172
pride - Train: 39009, Test: 9754
realization - Train: 41318, Test: 10330
relief - Train: 38953, Test: 9739
remorse - Train: 38994, Test: 9749
sadness - Train: 39765, Test: 9942
surprise - Train: 39513, Test: 9879
neutral - Train: 38756

In [ ]:
emotion_data.keys()

#Preparing the data for input into the model

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import StratifiedKFold

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, word_vectors, max_len=100):
        self.texts = texts
        self.labels = labels
        self.word_vectors = word_vectors
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        emotion_label = self.labels[idx]

        indexed_text = [self.word_vectors.get(word, np.zeros(300)) for word in text.split()]

        indexed_text = indexed_text[:self.max_len] if len(indexed_text) > self.max_len else indexed_text + [np.zeros(300)] * (self.max_len - len(indexed_text))

        indexed_text = torch.tensor(np.array(indexed_text), dtype=torch.float)
        emotion_label = torch.tensor(emotion_label, dtype=torch.float)

        return indexed_text, emotion_label

# Dictionary to store dataloaders for each emotion
emotion_dataloaders = {}

k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

for emotion in emotion_columns:
    train_texts = emotion_data[emotion]['train']['processed_text'].tolist()
    train_labels = emotion_data[emotion]['train'][emotion].tolist()

    test_texts = emotion_data[emotion]['test']['processed_text'].tolist()
    test_labels = emotion_data[emotion]['test'][emotion].tolist()

    test_dataset = EmotionDataset(test_texts, test_labels, word_vectors)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    emotion_dataloaders[emotion] = {'test': test_dataloader}

    X = np.array(train_texts)
    y = np.array(train_labels)

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"{emotion} - Training on Fold {fold+1}...")

        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        train_dataset_fold = EmotionDataset(X_train_fold, y_train_fold, word_vectors)
        val_dataset_fold = EmotionDataset(X_val_fold, y_val_fold, word_vectors)

        train_dataloader_fold = DataLoader(train_dataset_fold, batch_size=32, shuffle=True)
        val_dataloader_fold = DataLoader(val_dataset_fold, batch_size=32, shuffle=False)

        if 'folds' not in emotion_dataloaders[emotion]:
            emotion_dataloaders[emotion]['folds'] = []

        emotion_dataloaders[emotion]['folds'].append({'train': train_dataloader_fold, 'val': val_dataloader_fold})

print("Dataloaders ready for all emotions!")


admiration - Training on Fold 1...
admiration - Training on Fold 2...
admiration - Training on Fold 3...
admiration - Training on Fold 4...
admiration - Training on Fold 5...
amusement - Training on Fold 1...
amusement - Training on Fold 2...
amusement - Training on Fold 3...
amusement - Training on Fold 4...
amusement - Training on Fold 5...
anger - Training on Fold 1...
anger - Training on Fold 2...
anger - Training on Fold 3...
anger - Training on Fold 4...
anger - Training on Fold 5...
annoyance - Training on Fold 1...
annoyance - Training on Fold 2...
annoyance - Training on Fold 3...
annoyance - Training on Fold 4...
annoyance - Training on Fold 5...
approval - Training on Fold 1...
approval - Training on Fold 2...
approval - Training on Fold 3...
approval - Training on Fold 4...
approval - Training on Fold 5...
caring - Training on Fold 1...
caring - Training on Fold 2...
caring - Training on Fold 3...
caring - Training on Fold 4...
caring - Training on Fold 5...
confusion - Tra

In [ ]:
import torch
import torch.nn as nn

class EmotionLSTMWithAttention(nn.Module):
    def __init__(self, input_dim=300, hidden_dim=128, output_dim=1, n_layers=2, dropout=0.5):
        super(EmotionLSTMWithAttention, self).__init__()
        self.input_dropout = nn.Dropout(dropout)
        # Bidirectional LSTM Layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=n_layers, batch_first=True, dropout=dropout, bidirectional=True)

        # Attention Mechanism
        self.attention = nn.Linear(hidden_dim * 2, 1)  # Attention mechanism to calculate attention scores
        self.attn_dropout = nn.Dropout(dropout)
        # LayerNorm after LSTM
        self.ln = nn.LayerNorm(hidden_dim * 2)  # Since bidirectional, we multiply hidden_dim by 2

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        # Dropout after the LSTM and fully connected layer
        self.dropout = nn.Dropout(dropout)

    def attention_mechanism(self, lstm_output):
        """
        Applies attention mechanism over the LSTM outputs.
        """
        attention_scores = self.attention(lstm_output)  # Shape: (batch_size, seq_len, 1)
        attention_weights = torch.softmax(attention_scores, dim=1)  # Shape: (batch_size, seq_len, 1)
        weighted_sum = torch.sum(attention_weights * lstm_output, dim=1)  # Shape: (batch_size, hidden_dim*2)
        weighted_sum = self.attn_dropout(weighted_sum)
        return weighted_sum

    def forward(self, x):
        # x is the input tensor of shape (batch_size, seq_len, input_dim)
        x = self.input_dropout(x)
        # LSTM forward pass
        lstm_out, (hidden, cell) = self.lstm(x)

        # Apply attention mechanism
        attention_out = self.attention_mechanism(lstm_out)

        # Layer normalization
        attention_out = self.ln(attention_out)

        # Pass through fully connected layer and dropout
        out = self.fc(attention_out)
        out = self.dropout(out)

        return out

# Instantiate the model with the desired parameters
emotion_models = {}

for emotion in emotion_columns:
    model = EmotionLSTMWithAttention(input_dim=300, hidden_dim=128, output_dim=1, n_layers=2, dropout=0.5)
    emotion_models[emotion] = model

#white Head attention

In [ ]:
import torch
import torch.nn as nn

class EmotionLSTMWithAttention(nn.Module):
    def __init__(self, input_dim=300, hidden_dim=128, output_dim=1, n_layers=2, dropout=0.5,num_heads = 8):
        super(EmotionLSTMWithAttention, self).__init__()
        self.input_dropout = nn.Dropout(dropout)
        # Bidirectional LSTM Layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=n_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.ln_lstm = nn.LayerNorm(hidden_dim * 2)

        self.multihead_attention = nn.MultiheadAttention(hidden_dim * 2, num_heads=num_heads, dropout=dropout)  # Added multi-head attention

        # Attention Mechanism
        self.attention = nn.Linear(hidden_dim * 2, 1)  # Attention mechanism to calculate attention scores
        self.attn_dropout = nn.Dropout(dropout)
        # LayerNorm after LSTM
        self.ln = nn.LayerNorm(hidden_dim * 2)  # Since bidirectional, we multiply hidden_dim by 2

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        # Dropout after the LSTM and fully connected layer
        self.dropout = nn.Dropout(dropout)
        self.pos_weight = None


    def attention_mechanism(self, lstm_output):
        """
        Applies attention mechanism over the LSTM outputs.
        """
        attention_scores = self.attention(lstm_output)  # Shape: (batch_size, seq_len, 1)
        attention_weights = torch.softmax(attention_scores, dim=1)  # Shape: (batch_size, seq_len, 1)
        weighted_sum = torch.sum(attention_weights * lstm_output, dim=1)  # Shape: (batch_size, hidden_dim*2)
        weighted_sum = self.attn_dropout(weighted_sum)
        return weighted_sum

    def forward(self, x):
        # x is the input tensor of shape (batch_size, seq_len, input_dim)
        x = self.input_dropout(x)
        # LSTM forward pass
        lstm_out, (hidden, cell) = self.lstm(x)

        # Apply LayerNorm after LSTM
        lstm_out = self.ln_lstm(lstm_out)

        # Apply Multi-Head Attention
        attn_out, _ = self.multihead_attention(lstm_out, lstm_out, lstm_out)


        # Apply attention mechanism
        attention_out = self.attention_mechanism(attn_out)

        # Layer normalization
        attention_out = self.ln(attention_out)

        # Pass through fully connected layer and dropout
        out = self.fc(attention_out)
        out = self.dropout(out)

        return out

# Instantiate the model with the desired parameters
emotion_models = {}

for emotion in emotion_columns:
    model = EmotionLSTMWithAttention(input_dim=300, hidden_dim=128, output_dim=1, n_layers=2, dropout=0.3)
    emotion_models[emotion] = model

In [ ]:


# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Focal Loss Implementation
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2, reduction='mean', pos_weight=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.pos_weight = pos_weight

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight, reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        return F_loss.mean() if self.reduction == 'mean' else F_loss.sum()

# Training function with additional metrics, early stopping, and LR scheduler
def simple_train(model, train_dataloader_fold, val_dataloader_fold, criterion, optimizer, epochs=5):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    val_precisions = []
    val_recalls = []
    val_f1s = []
    val_aucs = []
    val_pr_auc = []

    best_loss = float('inf')
    epochs_without_improvement = 0
    patience = 3

    model.to(device)
    writer = SummaryWriter('runs/emotion_model')
    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        epoch_train_loss = 0
        epoch_train_acc = 0

        for inputs, labels in train_dataloader_fold:
            inputs, labels = inputs.to(device), labels.to(device).float()
            optimizer.zero_grad()

            outputs = model(inputs)
            labels = labels.float()
            loss = criterion(outputs.view(-1), labels)
            epoch_train_loss += loss.item()

            preds = torch.round(torch.sigmoid(outputs)).detach().cpu().numpy()
            batch_acc = f1_score(labels.detach().cpu().numpy(), preds, average="macro", zero_division=0)
            epoch_train_acc += batch_acc

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        avg_train_loss = epoch_train_loss / len(train_dataloader_fold)
        avg_train_acc = epoch_train_acc / len(train_dataloader_fold)
        train_losses.append(avg_train_loss)
        train_accuracies.append(avg_train_acc)

        model.eval()
        epoch_val_loss = 0
        epoch_val_acc = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_dataloader_fold:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                labels = labels.float()
                loss = criterion(outputs.view(-1), labels)
                epoch_val_loss += loss.item()

                preds = torch.round(torch.sigmoid(outputs)).detach().cpu().numpy()
                batch_acc = f1_score(labels.detach().cpu().numpy(), preds, average="macro", zero_division=0)
                epoch_val_acc += batch_acc

                all_preds.extend(preds)
                all_labels.extend(labels.detach().cpu().numpy())

        avg_val_loss = epoch_val_loss / len(val_dataloader_fold)
        avg_val_acc = epoch_val_acc / len(val_dataloader_fold)
        val_losses.append(avg_val_loss)
        val_accuracies.append(avg_val_acc)

        epoch_val_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
        epoch_val_precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        epoch_val_recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

        try:
            epoch_val_auc = roc_auc_score(all_labels, all_preds)
        except Exception:
            epoch_val_auc = 0.5

        val_f1s.append(epoch_val_f1)
        val_precisions.append(epoch_val_precision)
        val_recalls.append(epoch_val_recall)
        val_aucs.append(epoch_val_auc)

        epoch_time = time.time() - start_time
        print(f"Epoch {epoch+1}/{epochs} - Time: {epoch_time:.2f}s - Train Loss: {avg_train_loss:.4f} - Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), f'best_model_{label}.pt')
            model.to(device)
            print("  Saved the best model!")
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(" ⏹️ Early stopping triggered. Stopping training.")
            break

    return train_losses, val_losses, train_accuracies, val_accuracies, val_precisions, val_recalls, val_f1s, val_aucs

# Training loop for all emotion models
for label, model in emotion_models.items():
    print(f"Starting training for emotion: {label}")

    model.to(device)

    train_labels = emotion_data[label]['train'][label].tolist()
    val_labels = emotion_data[label]['test'][label].tolist()

    train_texts = emotion_data[label]['train']['processed_text'].tolist()
    val_texts = emotion_data[label]['test']['processed_text'].tolist()

    label_counts = np.bincount(train_labels)

    if len(label_counts) < 2:
        print(f"⚠️  Skipping training for {label} (only one class present)")
        continue

    weights = 1.0 / label_counts
    sample_weights = weights[train_labels]
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

    # pos_count = np.sum(train_labels)
    # neg_count = len(train_labels) - pos_count
    # pos_weight = torch.tensor([neg_count / max(pos_count, 1)]).to(device)

    pos_weight = torch.tensor([len(train_labels) - np.sum(train_labels)] / max(np.sum(train_labels), 1)).to(device)

    criterion = FocalLoss(alpha=0.20, gamma=2.5, pos_weight=pos_weight)
    train_dataset = EmotionDataset(train_texts, train_labels, word_vectors)
    val_dataset = EmotionDataset(val_texts, val_labels, word_vectors)

    train_dataloader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)

    best_loss = float('inf')
    epochs_without_improvement = 0
    patience = 3

    results = simple_train(model, train_dataloader, val_dataloader, criterion, optimizer, epochs=10)



#Test

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

os.makedirs("confusion_matrices", exist_ok=True)

def plot_confusion_matrix(y_true, y_pred, emotion):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7, 6))
    ax = sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", cbar=False, linewidths=1, linecolor='gray')

    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title(f'Confusion Matrix for {emotion}')

    plt.savefig(f'confusion_matrices/confusion_matrix_{emotion}.png', dpi=300)
    plt.close()

def evaluate_model_on_test(model, test_dataloader, emotion):
    model.eval()

    all_labels = []
    all_preds = []

    precision_list = []
    recall_list = []
    f1_list = []
    accuracy_list = []

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.round(torch.sigmoid(outputs)).cpu().numpy()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds)

            precision = precision_score(labels.cpu().numpy(), preds, zero_division=1)
            recall = recall_score(labels.cpu().numpy(), preds, zero_division=1)
            f1 = f1_score(labels.cpu().numpy(), preds, zero_division=1)
            accuracy = accuracy_score(labels.cpu().numpy(), preds)

            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)
            accuracy_list.append(accuracy)

    print(f"Evaluation results for {emotion}:")
    print(f"Precision: {np.mean(precision_list):.4f}")
    print(f"Recall: {np.mean(recall_list):.4f}")
    print(f"F1 Score: {np.mean(f1_list):.4f}")
    print(f"Accuracy: {np.mean(accuracy_list):.4f}")

    plot_confusion_matrix(np.array(all_labels), np.array(all_preds), emotion)

    return {
        'precision': np.mean(precision_list),
        'recall': np.mean(recall_list),
        'f1_score': np.mean(f1_list),
        'accuracy': np.mean(accuracy_list)
    }

all_metrics = {}

for emotion in emotion_columns:
    if emotion in combined_df.columns:
        train, test = split_data_for_emotion(combined_df, emotion_columns, emotion)

        test_dataloader = DataLoader(EmotionDataset(test['processed_text'].tolist(), test[emotion].tolist(), word_vectors), batch_size=32, shuffle=False)

        print(f"Evaluating model for {emotion}...")
        metrics = evaluate_model_on_test(trained_models[emotion], test_dataloader, emotion)

        all_metrics[emotion] = metrics
    else:
        print(f"Emotion {emotion} not found in the dataset.")

print("Evaluation complete for all emotions!")

def plot_comparison_graph(metrics_dict):
    emotions = list(metrics_dict.keys())
    precision = [metrics_dict[emotion]['precision'] for emotion in emotions]
    recall = [metrics_dict[emotion]['recall'] for emotion in emotions]
    f1 = [metrics_dict[emotion]['f1_score'] for emotion in emotions]
    accuracy = [metrics_dict[emotion]['accuracy'] for emotion in emotions]

    fig, ax = plt.subplots(2, 2, figsize=(16, 12))

    ax[0, 0].barh(emotions, precision, color='lightblue')
    ax[0, 0].set_title('Precision per Emotion')

    ax[0, 1].barh(emotions, recall, color='lightgreen')
    ax[0, 1].set_title('Recall per Emotion')

    ax[1, 0].barh(emotions, f1, color='lightcoral')
    ax[1, 0].set_title('F1 Score per Emotion')

    ax[1, 1].barh(emotions, accuracy, color='lightyellow')
    ax[1, 1].set_title('Accuracy per Emotion')

    plt.tight_layout()
    plt.savefig('confusion_matrices/model_comparison.png', dpi=300)
    plt.show()

plot_comparison_graph(all_metrics)


In [ ]:
!pip install ace_tools

In [ ]:
import torch
import torch.nn.functional as F
import pandas as pd
import os

emotion_models = {emotion: None for emotion in emotion_columns}


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def load_trained_models(emotion_models, model_class, device):
    trained_models = {}
    for emotion in emotion_models.keys():
        model_filename = f'best_model_{emotion}.pt'
        if os.path.exists(model_filename):
            model = model_class()
            model.load_state_dict(torch.load(model_filename, map_location=device))
            model.to(device)
            model.eval()
            trained_models[emotion] = model
        else:
            print(f"⚠️ Warning: Model file {model_filename} not found, skipping {emotion}")

    return trained_models

def predict_ensemble(text, trained_models, word_vectors, device):
    text_vectorized = [word_vectors.get(word, np.zeros(300)) for word in text.split()]
    text_vectorized = text_vectorized[:100] if len(text_vectorized) > 100 else text_vectorized + [np.zeros(300)] * (100 - len(text_vectorized))
    text_tensor = torch.tensor(np.array(text_vectorized), dtype=torch.float).unsqueeze(0).to(device)

    emotion_probs = {}
    for emotion, model in trained_models.items():
        with torch.no_grad():
            output = model(text_tensor)
            prob = torch.sigmoid(output).item()
            emotion_probs[emotion] = prob

    total_prob = sum(emotion_probs.values())
    for emotion in emotion_probs:
        emotion_probs[emotion] /= total_prob if total_prob > 0 else 1

    df = pd.DataFrame(list(emotion_probs.items()), columns=['Emotion', 'Probability'])
    df = df.sort_values(by='Probability', ascending=False)
    return df

text = "I feel so sad"
trained_models = load_trained_models(emotion_models, EmotionLSTMWithAttention, device)
result_df = predict_ensemble(text, trained_models, word_vectors, device)
display(result_df)

,Emotion,Probability
16,grief,0.048088
24,remorse,0.047397
25,sadness,0.047073
14,fear,0.047000
23,relief,0.046654
19,nervousness,0.046477
12,embarrassment,0.046186
17,joy,0.043750
11,disgust,0.043409
13,excitement,0.042442


#test white a new text's

In [ ]:
# List of sentences
texts = [
    #Negative emotions
    "I feel so sad",
    "This is the worst day of my life",
    "I'm feeling angry and frustrated",
    "Everything seems hopeless",
    "I'm so disappointed in myself",

    #Positive emotions
    "I'm so happy today!",
    "I feel loved and appreciated",
    "This is the best day ever",
    "I'm really excited about the future",
    "I feel so grateful for my friends",

    # Mixed emotions
    "I'm excited but also nervous about the presentation",
    "I'm happy to see you, but also feel a bit sad",
    "I feel proud but anxious about the exam results",
    "I'm happy but also a little scared about the changes",
    "I’m feeling joyful and overwhelmed at the same time",

    # Neutral emotions
    "I just woke up",
    "The weather is fine today",
    "I need to go to work",
    "The meeting is scheduled for 3 PM",
    "I'm reading a book right now",

    #Subtle emotions
    "I don’t know what to feel about this",
    "It’s hard to say how I feel",
    "I have mixed feelings about this",
    "I’m trying to understand my emotions",
    "It’s complicated"
]


trained_models = load_trained_models(emotion_models, EmotionLSTMWithAttention, device)

for text in texts:
    result_df = predict_ensemble(text, trained_models, word_vectors, device)
    print(f"Text: '{text}'")
    print(result_df)
    print("=" * 50)


Text: 'I feel so sad'
           Emotion  Probability
16           grief     0.048088
24         remorse     0.047397
25         sadness     0.047073
14            fear     0.047000
23          relief     0.046654
19     nervousness     0.046477
12   embarrassment     0.046186
17             joy     0.043750
11         disgust     0.043409
13      excitement     0.042442
9   disappointment     0.038526
5           caring     0.038389
26        surprise     0.037705
27         neutral     0.035936
2            anger     0.035315
3        annoyance     0.034298
0       admiration     0.032689
1        amusement     0.032202
4         approval     0.031459
20        optimism     0.029551
22     realization     0.029143
18            love     0.028479
8           desire     0.027091
15       gratitude     0.026203
7        curiosity     0.024466
10     disapproval     0.024220
6        confusion     0.022398
21           pride     0.013454
Text: 'This is the worst day of my life'
         

#white llm

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F

# Tokenizer and model for BERT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BERT model with 28 output labels (for 28 emotions)
bert_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=28).to(device)

# Function to predict using BERT for multiple emotions and apply Softmax
def predict_bert(text, tokenizer, bert_model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1).squeeze().cpu().numpy()  # Apply Softmax to logits
    return probs

# Example text for prediction
text = "I feel so sad"
probs_bert = predict_bert(text, tokenizer, bert_model, device)

# Emotion labels for the 28 emotions
emotion_labels = emotion_columns

# Create a DataFrame with the emotions and BERT's probabilities
df_bert_predictions = pd.DataFrame({
    'Emotion': emotion_labels,
    'BERT Probability': probs_bert
})

# Get the emotion with the highest probability
df_bert_predictions['Max Probability'] = df_bert_predictions['BERT Probability'].max()
df_bert_predictions['BERT Prediction'] = np.where(df_bert_predictions['BERT Probability'] == df_bert_predictions['Max Probability'], 1, 0)

# Sort by the highest probability
df_bert_predictions = df_bert_predictions.sort_values(by='BERT Probability', ascending=False)

# Show the final predictions for BERT
print(df_bert_predictions)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


           Emotion  BERT Probability  Max Probability  BERT Prediction
23          relief          0.051065         0.051065                1
14            fear          0.049554         0.051065                0
7        curiosity          0.049232         0.051065                0
15       gratitude          0.047803         0.051065                0
1        amusement          0.044621         0.051065                0
11         disgust          0.043542         0.051065                0
2            anger          0.043074         0.051065                0
6        confusion          0.042843         0.051065                0
27         neutral          0.041821         0.051065                0
18            love          0.040525         0.051065                0
10     disapproval          0.038225         0.051065                0
20        optimism          0.037677         0.051065                0
16           grief          0.035970         0.051065                0
24    

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

# Tokenizer and model for BERT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to load trained models
def load_trained_models(emotion_models, model_class, device):
    trained_models = {}
    for emotion in emotion_models:  # Iterate over the list of emotions
        model_filename = f'best_model_{emotion}.pt'
        if os.path.exists(model_filename):
            model = model_class()
            model.load_state_dict(torch.load(model_filename, map_location=device))
            model.to(device)
            model.eval()
            trained_models[emotion] = model
        else:
            print(f"⚠️ Warning: Model file {model_filename} not found, skipping {emotion}")

    return trained_models

# Function to predict using the ensemble of models
def predict_ensemble(text, trained_models, word_vectors, device):
    text_vectorized = [word_vectors.get(word, np.zeros(300)) for word in text.split()]
    text_vectorized = text_vectorized[:100] if len(text_vectorized) > 100 else text_vectorized + [np.zeros(300)] * (100 - len(text_vectorized))
    text_tensor = torch.tensor(np.array(text_vectorized), dtype=torch.float).unsqueeze(0).to(device)

    emotion_probs = {}
    for emotion, model in trained_models.items():
        with torch.no_grad():
            output = model(text_tensor)
            prob = torch.sigmoid(output).item()
            emotion_probs[emotion] = prob

    total_prob = sum(emotion_probs.values())
    for emotion in emotion_probs:
        emotion_probs[emotion] /= total_prob if total_prob > 0 else 1

    df = pd.DataFrame(list(emotion_probs.items()), columns=['Emotion', 'Probability'])
    df = df.sort_values(by='Probability', ascending=False)
    return df

# Function to calculate cosine similarity
def calculate_cosine_similarity(probs_1, probs_2):
    return cosine_similarity([probs_1], [probs_2])[0][0]

# Function to calculate Euclidean distance
def calculate_euclidean_distance(probs_1, probs_2):
    return np.linalg.norm(np.array(probs_1) - np.array(probs_2))

# Load BERT model
bert_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name).to(device)

# Function to predict using BERT
def predict_bert(text, tokenizer, bert_model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
    return probs

# Example text for prediction
text = "I feel so sad"

# Calculate probabilities for your ensemble model
trained_models = load_trained_models(emotion_models, EmotionLSTMWithAttention, device)
result_df = predict_ensemble(text, trained_models, word_vectors, device)
emotion_probs_ensemble = result_df['Probability'].values  # Probabilities from the ensemble

# Calculate probabilities for BERT
probs_bert = predict_bert(text, tokenizer, bert_model, device)


bert_probs_adjusted = np.zeros(28)  # Create a 28-length array
bert_probs_adjusted[:2] = probs_bert  # Assign the first 2 probabilities from BERT

# Now calculate cosine similarity and Euclidean distance using the same dimensions
cosine_sim = calculate_cosine_similarity(emotion_probs_ensemble, bert_probs_adjusted)
euclidean_dist = calculate_euclidean_distance(emotion_probs_ensemble, bert_probs_adjusted)

# Output the results
print(f"Cosine Similarity: {cosine_sim}")
print(f"Euclidean Distance: {euclidean_dist}")

# Combine the probabilities from the ensemble model and BERT
combined_probs = np.vstack([emotion_probs_ensemble, bert_probs_adjusted]).T  # Stack them side by side

# Create the DataFrame with appropriate column names
df_comparison = pd.DataFrame(combined_probs, columns=['Ensemble Probability', 'BERT Probability'])

# Optionally, if you want to see which emotions BERT is matching
df_comparison['Emotion'] = result_df['Emotion'].values

# Display the DataFrame
print(df_comparison)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Cosine Similarity: 0.3459254372478616
Euclidean Distance: 0.7708719591225267
           Emotion  Ensemble Prediction  BERT Prediction
0            grief                    0                1
1          remorse                    0                1
2          sadness                    0                0
3             fear                    0                0
4           relief                    0                0
5      nervousness                    0                0
6    embarrassment                    0                0
7              joy                    0                0
8          disgust                    0                0
9       excitement                    0                0
10  disappointment                    0                0
11          caring                    0                0
12        surprise                    0                0
13         neutral                    0                0
14           anger                    0                0
15       an

In [ ]:
from transformers import AutoModelForSequenceClassification
from torch.utils.data import Dataset

class EmotionTextDataset(Dataset):
    def __init__(self, texts, emotion_probs, tokenizer):
        self.texts = texts
        self.emotion_probs = emotion_probs
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        emotion_prob = torch.tensor(self.emotion_probs[idx], dtype=torch.float)
        # Tokenize the text
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        # We return the input ids and attention mask together with the emotion probabilities
        return inputs['input_ids'].squeeze(0), emotion_prob



class EmotionAugmentedLLM(nn.Module):
    def __init__(self, llm_model_name, emotion_models):
        super(EmotionAugmentedLLM, self).__init__()
        # Load the large language model (LLM) for sequence classification
        self.llm = AutoModelForSequenceClassification.from_pretrained(llm_model_name)
        # Store the emotion models
        self.emotion_models = emotion_models
        # Add a layer to combine the LLM output with emotion probabilities
        self.emotion_fc = nn.Linear(self.llm.config.hidden_size + len(emotion_models), 2)  # Example binary classification

    def forward(self, text, emotion_features):
        # Pass text through the LLM model
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
        llm_output = self.llm(**inputs)
        # Concatenate LLM output with emotion features
        combined_features = torch.cat((llm_output.logits, emotion_features), dim=-1)
        # Pass combined features through a fully connected layer
        output = self.emotion_fc(combined_features)
        return output



def get_emotion_probabilities(text, emotion_models):
    emotion_probs = []
    for model in emotion_models:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
        emotion_prob = model(inputs['input_ids'], inputs['attention_mask']).squeeze().cpu().detach().numpy()
        emotion_probs.append(emotion_prob)
    return torch.tensor(emotion_probs).unsqueeze(0).to(device)  # [1, num_emotions]

# Instantiate models
llm_model_name = "bert-base-uncased"  # Example LLM (you can choose a different one)
emotion_models = [EmotionLSTMWithAttention() for _ in range(5)]  # Replace with real emotion models

# Create the Emotion-AUG LLM model
emotion_augmented_llm = EmotionAugmentedLLM(llm_model_name, emotion_models).to(device)

# Example text and emotion probabilities (for demo)
texts = ["I am so happy today!", "I feel so sad..."]
emotion_probs = [
    [0.9, 0.05, 0.1, 0.2, 0.05],  # Example emotion probabilities for the first text
    [0.1, 0.9, 0.2, 0.5, 0.05]   # Example emotion probabilities for the second text
]

# Prepare dataset and dataloader
dataset = EmotionTextDataset(texts, emotion_probs, tokenizer=AutoTokenizer.from_pretrained(llm_model_name))
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

# Training loop
optimizer = optim.Adam(emotion_augmented_llm.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    emotion_augmented_llm.train()
    running_loss = 0.0
    for text, emotion_labels in dataloader:
        optimizer.zero_grad()

        # Get emotion probabilities
        emotion_features = get_emotion_probabilities(text, emotion_models)

        # Forward pass
        outputs = emotion_augmented_llm(text, emotion_features)

        # Calculate loss and backpropagate
        loss = criterion(outputs, emotion_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/10], Loss: {running_loss/len(dataloader)}")

# Save the model
torch.save(emotion_augmented_llm.state_dict(), "emotion_augmented_llm.pth")